<a href="https://colab.research.google.com/github/NichaB/LAW-Project/blob/main/OpenThaiGPT_Thai_GPT_2_Finetuning_for_OpenThaiGPT_0_0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! nvidia-smi

Sat Jul  8 03:46:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    46W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! pip install transformers[sentencepiece] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.3 MB/s eta 0:00:00


In [ ]:
# Restart Runtime NOW!

In [ ]:
#! huggingface-cli login

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel

pretrained_name = "flax-community/gpt2-base-thai"

tokenizer = GPT2Tokenizer.from_pretrained(pretrained_name, bos_token='<|startoftext|>',unk_token='<|unk|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
model = GPT2LMHeadModel.from_pretrained(pretrained_name).cuda()
model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(50268, 768)

In [ ]:
tokenizer

GPT2Tokenizer(name_or_path='flax-community/gpt2-base-thai', vocab_size=50265, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|startoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|unk|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<|pad|>", rstrip=False, lstrip=False, single_word=False, normalized=True)}, clean_up_tokenization_spaces=True)

In [ ]:
prompt = "<|startoftext|> 1+1 ได้เท่าไร่ <|endoftext|>"
encoded_input = tokenizer(prompt, return_tensors='pt')

In [ ]:
encoded_input


{'input_ids': tensor([[50266,   388,    15,    21,   370,   274,   447,   271,  4222,   271,
           225, 50265]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [ ]:
generated = tokenizer("<|startoftext|> ลดน้ำหนักต้องทำอย่างไร", return_tensors="pt").input_ids.cuda()
sample_outputs = model.generate(generated, do_sample=True, top_k=50,  max_length=300, top_p=0.95, temperature=1.9, num_return_sequences=20)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

0:  ลดน้ำหนักต้องทำอย่างไร เรื่องง่าย หรือของที่ก
ชิสุกี้หิวคงจะดีนะสัดส่วนอุ้งมือนั่นดูดีเหมือนฝาม็อบพุงติดสเก็ตจิ๊กซอร์วิ่งไม่หละไม่กี่วิ!! 12 ประโยคที่ได้ใจความว่ามันมันอยู่เหนือศีรษะคุณพ่อมาแล้วในภาษาของเราที่มากมายทั้งๆทักษะมีการทดสอบเพื่อใส่รองเท้าไปยูเวนตุสเตอบอกเขาจบดูถูกใครบางคนและกางเกงขาก๊วนน่ามอง
ความแตกต่างกันและผลเล็กๆน้อยๆจากยัยหนูเดี๋ยวรุ่ง!!!และมรดกจากญี่ปุ่นเปิดเผ่ากันมานาฬใบโปรดจะได้รับยาตลอดไป คึกอย่าคุยกับสิงสาราสังข์ไม่เว้นรายการก็จะเกิดช่วงหลับยาวหรือแม้แต่สัตว์สองบ้านคั่นอยู่ในท่วงทำนองของน้ำถ
ผักสดฝึกหนาวมา!สิ่งเสพติดนิ่วจากถั่วในช่วงเดือนกวนหนืดลงมาต
1:  ลดน้ำหนักต้องทำอย่างไรมาชื่นต่อเนื่องกับข้าวคนไทยรักกุ้งยังไง ว่าแล้วเป็นเราจะลงสมัครเรียนอาชีวะคิดไปตั้งหลัง ริเริ่มแต่ตอนยังกับหัววันใหม่ ๆ จุลินทรียจากซันเดย์แคมบ์,จึงเป็นจู่ถาวรสวยอู้ชิ้นจนน้อยไปจึงเปื้อนเทอ ทันทีและคืนสวยกลีบอย่างชัดเจนว่า ชั้นไวหนึง,บิวตี้พคู่หาพุง แตกกรงไดแล้วเด็กคืนว้อง และตีนบาตรพิริปเยี่ยมอย่างของกาลกินาแล้วบ๊วยพรสชาติกุ้งเปนต้องการแลกทั้งหด ตัวเล่นน้ำยืนมันถั่วอยุน,ห่อตัวแต่งภอแลกกิ

# Pantip

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!gdown 10A8DbHItymMiPIQP7zxAuDKujDIv4DRR

Downloading...
From: https://drive.google.com/uc?id=10A8DbHItymMiPIQP7zxAuDKujDIv4DRR
To: /content/Pantip_QA_200000_20220220.csv
100% 738M/738M [00:06<00:00, 113MB/s]


In [ ]:
torch.manual_seed(42)


In [ ]:
pantip = pd.read_csv('Pantip_QA_200000_20220220.csv')


In [ ]:
pantip = pantip.dropna()

In [ ]:
pantip

,Unnamed: 0,topic_id,prompt,comment
0,0,41872704,"""กาลเวลาอาจมีเงื่อนไข แต่แตกต่างกับใจของฉัน ที...",สิ่งใหม่ใหม่ ไม่เคยให้เข้ามา\nคนไม่มีเวลา แม้น...
1,1,41872704,"""กาลเวลาอาจมีเงื่อนไข แต่แตกต่างกับใจของฉัน ที...",ความทรงจำที่ดี ฉันขอให้มัน ไม่หายไป\nแต่เธอคงจ...
2,2,41872704,"""กาลเวลาอาจมีเงื่อนไข แต่แตกต่างกับใจของฉัน ที...",เจ็บปวดรู้ไหม ไม่อยากเคลื่อนไหว\n\nไม่อยากทำร้...
3,3,41872704,"""กาลเวลาอาจมีเงื่อนไข แต่แตกต่างกับใจของฉัน ที...",เธอ...อยากกลับมาไหม\n\nอยู่ตัวคนเดียว ไม่เหลือ...
4,4,41872703,ยานอนหลับชนิดไหนดี ช่วงนี้แฟนผมออกห่างผมเลยถาม...,คงต้องปรึกษาแพทย์
...,...,...,...,...
294673,294673,41806467,ขอรบกวนสอบถาม พี่ๆที่ติดตามวงไอดอลของไทย มาหลา...,ASTERส่วนใหญ่คนที่ตามตอนนั้นก็ตามจากdaisy dais...
294674,294674,41806467,ขอรบกวนสอบถาม พี่ๆที่ติดตามวงไอดอลของไทย มาหลา...,แมวสยาม รุ่นแรกเคยดังในโลกโซเชียลอยู่ช่วงนึงใน...
294675,294675,41806467,ขอรบกวนสอบถาม พี่ๆที่ติดตามวงไอดอลของไทย มาหลา...,แอสเทอ ดังระดับนึง ออกงานบ่อยเสตจเป็นเอกลักษณ์...
294676,294676,41806467,ขอรบกวนสอบถาม พี่ๆที่ติดตามวงไอดอลของไทย มาหลา...,แอสเทอ ดังระดับนึง ออกงานบ่อยเสตจเป็นเอกลักษณ์...


In [ ]:
pantip["prompt"][0]

'"กาลเวลาอาจมีเงื่อนไข แต่แตกต่างกับใจของฉัน ที่จะรอเพียงเธอ...เหมือนเดิม" ฉากเก่าเก่า ยังจำไว้ในใจ\nเคยรักเธอยังไง รักเธอดังเดิม\nภาพทุกภาพ ยังตามติดอยู่ ไม่หายไป\nเสียงหัวเราะบนความอ่อนไหว\n\nเจ็บปวดรู้ไหม ไม่อยากเคลื่อนไหว\nไม่อยากทำร้ายใคร\nมันยังไม่พร้อมมีใครใช่ไหม\nจะขอรอเธอ อย่างเดิม\n\nเธอ...อยากกลับมาไหม\nอยู่ตัวคนเดียว ไม่เหลือใคร\nเหงาเกินไปบ้างไหมเธอ...\n\nรอ...อยู่ไม่ไปไหน\nปล่อยให้เวลา ผ่านฉันไป\nเพราะลืมเธอไม่ไหว ไม่ได้อยู่ดี'

In [ ]:
pantip["comment"][1]

'ความทรงจำที่ดี ฉันขอให้มัน ไม่หายไป\nแต่เธอคงจะลืม เมื่อเธอมีใคร\n\nกาลเวลา อาจมีเงื่อนไข\nแต่แตกต่าง กับใจของฉัน\nที่จะรอเพียงเธอ...เหมือนเดิม...\n\nยังรอ อยู่ไม่ไปไหน\nปล่อยให้เวลาผ่านฉันไป\nฉันลืมเธอไม่ไหว ไม่ได้อยู่ดี...'

In [ ]:
import re

In [ ]:
pantip_texts = []
for (idx, row) in pantip.iterrows():
  tmp = "Q: "+row["prompt"][:512]+"\n\nA: "+row["comment"]
  if (tmp!=None and len(tmp)>10):
    cleaned = re.sub(r'[^\u0E00-\u0E7F\u0020-\u007E0-9\n]', '', str(tmp))
    pantip_texts.append(cleaned)

In [ ]:
pantip_texts[102]

'Q: เมนูแบบนี้คุณกินไหมครับ แล้วกุ้งเต้นล่ะครับ? คุณกินไหม?\nกินไปดิ้นไป\n\nA: ปลาปกติก็กินแล้วกลัวก้างติดคออยู่แล้ว เจอแบบนี้ไม่มีทางกินเลยค่ะ'

In [ ]:
pantip_texts = pantip_texts[:3000]

In [ ]:
len(pantip_texts)

3000

# Thai QA

In [ ]:
# !pip install datasets

In [ ]:
# from datasets import load_dataset
# dataset = load_dataset("thaiqa_squad")

In [ ]:
# dataset

In [ ]:
# import pandas as pd
# qas = pd.DataFrame( dataset["train"] )

In [ ]:
# qas

In [ ]:
# qas['context'][0]

In [ ]:
# qas['question'][0]

In [ ]:
# qas['answers'][0]

In [ ]:
# import re
# qas_text = []

# for (idx, row) in qas.iterrows():
#   answers = row["answers"]
#   context_text = re.sub(r"(<.*?>)", "", row['context'])[:500]
#   question_text = row['question']
#   answer_text = answers["answer"][0]
#   text = f"{context_text}\n\nQ:{question_text}\n\nA:{answer_text}"
#   qas_text.append(text)

In [ ]:
# qas_text

# Concat

In [ ]:
pantip_texts

['Q: "กาลเวลาอาจมีเงื่อนไข แต่แตกต่างกับใจของฉัน ที่จะรอเพียงเธอ...เหมือนเดิม" ฉากเก่าเก่า ยังจำไว้ในใจ\nเคยรักเธอยังไง รักเธอดังเดิม\nภาพทุกภาพ ยังตามติดอยู่ ไม่หายไป\nเสียงหัวเราะบนความอ่อนไหว\n\nเจ็บปวดรู้ไหม ไม่อยากเคลื่อนไหว\nไม่อยากทำร้ายใคร\nมันยังไม่พร้อมมีใครใช่ไหม\nจะขอรอเธอ อย่างเดิม\n\nเธอ...อยากกลับมาไหม\nอยู่ตัวคนเดียว ไม่เหลือใคร\nเหงาเกินไปบ้างไหมเธอ...\n\nรอ...อยู่ไม่ไปไหน\nปล่อยให้เวลา ผ่านฉันไป\nเพราะลืมเธอไม่ไหว ไม่ได้อยู่ดี\n\nA: สิ่งใหม่ใหม่ ไม่เคยให้เข้ามา\nคนไม่มีเวลา แม้นานยังไง\nแค่พรุ่งนี้วันเดียว ที่ตื่นมาหายใจ\nและเพื่อพร้อมรอเธอต่อไป\n\nเหน็ดเหนื่อยรู้ไหม ไม่อยากเคลื่อนไหว\nไม่อยากจะรักใคร\nมันยังไม่พร้อมมีใครใช่ไหม\nจะขอรักเธอ อย่างเดิม',
 'Q: "กาลเวลาอาจมีเงื่อนไข แต่แตกต่างกับใจของฉัน ที่จะรอเพียงเธอ...เหมือนเดิม" ฉากเก่าเก่า ยังจำไว้ในใจ\nเคยรักเธอยังไง รักเธอดังเดิม\nภาพทุกภาพ ยังตามติดอยู่ ไม่หายไป\nเสียงหัวเราะบนความอ่อนไหว\n\nเจ็บปวดรู้ไหม ไม่อยากเคลื่อนไหว\nไม่อยากทำร้ายใคร\nมันยังไม่พร้อมมีใครใช่ไหม\nจะขอรอเธอ อย่างเดิม\n\nเธอ...อยากกลับมาไหม\nอย

In [ ]:
#qas_text

In [ ]:
all_text = pantip_texts #+ qas_text

In [ ]:
# all_text

In [ ]:
len(all_text)

3000

In [ ]:
# max_length = max([len(tokenizer.encode(text)) for text in texts])
max_length = 1024

In [ ]:
import numpy as np
import torch

In [ ]:
class ThaiDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            encodings_dict = tokenizer('<|startoftext|>' + txt + '<|endoftext|>', truncation=True,
                                      max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]


In [ ]:
dataset = ThaiDataset(all_text, tokenizer, max_length=max_length)
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

In [ ]:
import gc
gc.collect()

0

In [ ]:
torch.cuda.empty_cache()


In [ ]:
training_args = TrainingArguments(output_dir='./results', num_train_epochs=5, logging_steps=10, save_steps=100,
                                  per_device_train_batch_size=12, per_device_eval_batch_size=12,
                                  warmup_steps=10, weight_decay=0.05, logging_dir='./logs', report_to = 'none')

In [ ]:
trainer = Trainer(model=model,  args=training_args, train_dataset=train_dataset,
        eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                              'attention_mask': torch.stack([f[1] for f in data]),
                                                              'labels': torch.stack([f[0] for f in data])})
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,1.207800
20,1.040700
30,0.935700
40,0.805500
50,0.759000
60,0.679000
70,0.636300
80,0.735300
90,0.747000
100,0.658600


TrainOutput(global_step=1125, training_loss=0.5262068650987414, metrics={'train_runtime': 982.5887, 'train_samples_per_second': 13.739, 'train_steps_per_second': 1.145, 'total_flos': 7054884864000000.0, 'train_loss': 0.5262068650987414, 'epoch': 5.0})

# Testing Generation

In [ ]:
generated = tokenizer("<|startoftext|> Q: ลดน้ำหนักต้องทำอย่างไร\n\nA:", return_tensors="pt").input_ids.cuda()


In [ ]:
sample_outputs = model.generate(generated, do_sample=True, top_k=50, num_beams=5, no_repeat_ngram_size=2,
    early_stopping=True, max_length=300, top_p=0.95, temperature=1.9, num_return_sequences=20)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

0:  Q: ลดน้ำหนักต้องทำอย่างไร

A: เรามีแต่บุคคลที่สร้างสรรค์สังคม ทุกวันนี้ก็อยากขับให้สูงขึ้น ส่วนที่ไม่อยากได้มันก็ใช้คำว่ามันขึ้นมาแทน กับการเล่นกีฬาสูงๆนั้น
ผู้สื่อข่าวบ้านเราเข้าไปถามด้วย ถ้าตั้งใจอยู่แล้ว จะไปออกมาพูดเรื่องอื่นไม่รู้เรื่องส่วนใหญ่จะอ่านไปดูภาพ
ส่วนนี้ถือเป็นการแสดงพลังเยาวชนในวัฒนธรรมไทยก่อนที่จะมาถ่วงบริหารกองทุนที่อ้างถึงมันมีการถ่วงน้ำเนื้อหาในการใช้บังคับของนักลงทุนได้เขียนเมื่อ: บริการสุดท้ายก็ยังสามารถมีที่นี้มาเป็นข่าวทั่วไปท่องเที่ยวประเทศไทยธุรกิจสีลม ที่นอนเด็กเล็ก และอีกหลายคนบอกว่าคนทุกท่านมีปัจจัยการลงทุนแตกต่างจากประเทศไทย
นอก�
1:  Q: ลดน้ำหนักต้องทำอย่างไร

A: สำหรับผมวันนี้จะได้ไม่ค่อยมาอธิบายครับ เพราะสิ่งที่เราจะเอามาคืออาการของโรคต่างๆที่เกิดขึ้นจากภายในร่างกายของเรา เราจะรู้ว่ามันจะมาจากไหน ต้องถามด้วยว่า
ส่วนวีรกรรมสุดท้ายก่อนเลือกข้างโรงเรียน มีบัตรปีแรกของการให้บริหารจัดการศึกษาแห่งชาติ ส่วนที่จะเลือกตั้งแต่แรกจะเป็นบทพิเศษ
มีอะไรมันยังไงอยู่ตรงนี้ ไม่ใช่ประเด็นหลักๆอยู่แล้ว
ผู้ใหญ่ในวงการพระพุทธศาสนาห้ามเข้าใจผิด พิธีกรทุกวัน อย่างเดียว มั

# Save to Hugging Face

In [ ]:
# model.push_to_hub("openthaigpt-gpt2-pantipwiki-poc")
# tokenizer.push_to_hub("openthaigpt-gpt2-pantipwiki-poc")

---

In [ ]:
# from transformers import GPT2LMHeadModel, GPT2Tokenizer
# import pandas as pd
# import torch
# from torch.utils.data import Dataset, random_split
# from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel

# pretrained_name = "kobkrit/openthaigpt-gpt2-pantipwiki-poc"

# tokenizer = GPT2Tokenizer.from_pretrained(pretrained_name, bos_token='<|startoftext|>',unk_token='<|unk|>', eos_token='<|endoftext|>', pad_token='<|pad|>')
# model = GPT2LMHeadModel.from_pretrained(pretrained_name).cuda()
# model.resize_token_embeddings(len(tokenizer))

In [ ]:
# generated = tokenizer("<|startoftext|>Q: สวัสดีครับ ผมมีแฟนน่ารักมากทำอย่างไรให้ไม่หายไปครับ\n\nA: ", return_tensors="pt").input_ids.cuda()
# sample_outputs = model.generate(generated, do_sample=True, top_k=50, num_beams=5, no_repeat_ngram_size=2,
#     early_stopping=True, max_length=300, top_p=0.95, temperature=1.9, num_return_sequences=5)

In [ ]:
# for i, sample_output in enumerate(sample_outputs):
#     print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))